### Instalacja

In [ ]:
%cd /content/
!git clone https://github.com/Bartolo72/phisher
%cd /content/phisher
!pip install -r requirements.txt -q
!pip install -e . -q
%cd /content/

### XAI

In [1]:
from phisher.xai import GradCAM
from phisher.module import PhisherhModule
from phisher.model import PhisherEmbeddingModel
from phisher.dataset import PhishingEmbeddingDataset

/Users/bkosinski/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

#### Load model

In [6]:
model_ckpt_path: str = "/Users/bkosinski/Desktop/STUDIA/SEMI/GSN/projekt/checkpoints/embeddings_model.ckpt"

In [7]:
model: PhisherEmbeddingModel = PhisherEmbeddingModel(vocab_size=84, embedding_dim=100, out_features=1)

In [8]:
PhisherhModule.load_from_checkpoint(model_ckpt_path, model=model)

/Users/bkosinski/Library/Python/3.9/lib/python/site-packages/lightning_fabric/utilities/cloud_io.py:55: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/Users/bkosinski/Library/Python/3.9/lib/

PhisherhModule(
  (model): PhisherEmbeddingModel(
    (embedding): Embedding(84, 100, padding_idx=0)
    (conv1): Conv2d(1, 6, kernel_size=(5, 1), stride=(1, 1))
    (conv2): Conv2d(6, 12, kernel_size=(5, 1), stride=(1, 1))
    (fc1): Linear(in_features=56400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=60, bias=True)
    (out): Linear(in_features=60, out_features=1, bias=True)
  )
  (accuracy): BinaryAccuracy()
  (f1): BinaryF1Score()
  (precision): BinaryPrecision()
  (recall): BinaryRecall()
  (auroc): BinaryAUROC()
)

#### Visualize Grad-CAM

In [9]:
def visualize_url_with_cam(url: str, cam: torch.Tensor) -> str:
    cam = cam.mean(dim=1).squeeze()
    cam = cam.cpu().detach().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min())
    url = url[: len(cam)]

    colored_url = ""
    for char, intensity in zip(url, cam):
        color_intensity = int(
            255 * (1 - intensity)
        )  # Scale intensity (higher = darker)
        hex_color = f"{color_intensity:02x}ff{color_intensity:02x}"
        colored_url += f'<span style="background-color: #{hex_color};">{char}</span>'

    return colored_url

In [28]:
device = torch.device("mps")

In [29]:
model = model.to(device)

In [61]:
grad_cam = GradCAM(model, target_layer=model.conv1)

In [89]:
placeholder_dataset = PhishingEmbeddingDataset(csv_file_path="/Users/bkosinski/Desktop/STUDIA/SEMI/GSN/projekt/phish_dataset.csv")

In [101]:
target_url: str = "booking.guest7confirmation.com"

In [102]:
input_tensor: torch.Tensor
input_tensor = placeholder_dataset.parse_url(target_url)
input_tensor = placeholder_dataset.pad_or_trim(input_tensor)
input_tensor = torch.tensor(input_tensor, dtype=torch.long)

input_tensor = input_tensor.to(device)
input_tensor = input_tensor.reshape(1, -1)

In [103]:
cam = grad_cam.generate_cam(input_tensor, target_class=0)
visualized_url: str = visualize_url_with_cam(url=target_url, cam=cam)

In [104]:
from IPython.core.display import display, HTML
display(HTML(f"<div style='color: black'>{visualized_url}</div>"))

/var/folders/l3/hg12zsjx6rsbsylxwh747cv00000gn/T/ipykernel_8486/1759908886.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
